# Implementing AI Guardrails

In this lab, you will implement guardrails for a simple generative AI application to secure it against malicious behavior and harmful generated output.


**Lab Outline:**

In this lab, you will need to complete the following tasks:

* **Task 1:** Implement LLM-based guardrails with Llama Guard model.
  1. Set Up Llama Guard Model and Configuration Variables
  2. Implement the query_llamaguard Function
  3. Test the Implementation
* **Task 2:** Customize Llama Guard Guardrails
  1. Define Custom Unsafe Categories
  2. Test the Implementation
* **Task 3:** Integrate Llama Guard with Chat Model
  1. Set up an non-Llama Guard query function
  2. Set up a Llama Guard query function
  3. Test the Implementation

In [0]:
%pip install -U -qq databricks-sdk mlflow
dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-02


The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


##Task 1: Implement LLM-based Guardrails with `Llama Guard`
First, To set up the safety measures of your application, you will integrate Llama Guard, a specialized model available on the Databricks Marketplace. This will enable you to classify chat content as safe or unsafe, allowing for more effective management of potentially harmful interactions.


**Llama Guard in Databricks**

To streamline the integration process and leverage the benefits of Llama Guard, a deployment of this model is readily available on the Databricks Marketplace.

**Instructions (To be performed by the instructor only):**

1. Find the "Llama Guard Model" in **Databricks Marketplace**.
2. Click on **Get Instant Access** to load it to a location in Unity Catalog.
3. **Deploy the model** to a Model Serving endpoint.

By integrating the Model Serving endpoint into your own application, you gain the flexibility to specify your own policies for detecting and preventing various types of content. This ensures that your application maintains a safe and secure environment for users.

**🚨 Warning:** Please avoid deploying the model yourself, as it may take time and might not be practical in a classroom setting. Instead, utilize the model that has been provided for this lab. For self-paced learners, you can follow the steps above to create the endpoint.


**Instructions:**
1. **Set Up Llama Guard Model and Configuration Variables**

    Set up a starting point description of your **Unsafe** category:

    * **To do:**
    
        You have been provided with 2 categories. Your task is to write descriptions for 3 additional categories based on common themes or topics. 

    
        **Hint:** You can use AI Playground to generate descriptions for these categories, ensuring they are relevant and distinct from the provided examples.

In [0]:
DEFAULT_UNSAFE_CATEGORY = """O1: Unsafe Content.
    - Promoting or assisting in violence or hate.
    - Engaging in or encouraging sexually explicit content.
    - Assisting or encouraging criminal activities, including those related to firearms or illegal weapons or robbery.
    - Encouraging or assisting in the use or production of illegal or controlled substances.
    - Encouraging or detailing self-harm or suicide."""

Define the necessary variables and set up the **Llama Guard** model.

In [0]:
## llama-guard endpoint name. Change this to your own endpoint name, if you manually create one!
LLAMAGUARD_ENDPOINT_NAME="llama-guard"

2. **Implement the `query_llamaguard` Function**

Develop a function to query the Llama Guard model and classify chat content as safe or unsafe.

In [0]:
from databricks.sdk import WorkspaceClient

def query_llamaguard(chat, unsafe_categories = DEFAULT_UNSAFE_CATEGORY):
    try:
        prompt = f"""[INST] Task: Check for unsafe content in 'Agent' messages in conversations according to our safety policy.
        <BEGIN UNSAFE CONTENT CATEGORIES>
        {unsafe_categories}
        <END UNSAFE CONTENT CATEGORIES>
        <BEGIN CONVERSATION>
        {chat}
        <END CONVERSATION>
        Provide your safety assessment for Agent in the above conversation:
        - First line must read 'safe' or 'unsafe'.
        - If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""

        w = WorkspaceClient()
        response = w.serving_endpoints.query(
            name=LLAMAGUARD_ENDPOINT_NAME, 
            prompt=prompt,
            temperature=0.1,
            max_tokens=128)
        
        ## Extract the desired information from the response object
        prediction = response.as_dict()["choices"][0]["text"].strip()
        is_safe = None if len(prediction.split("\n")) == 1 else prediction.split("\n")[1].strip()
        
        return prediction.split("\n")[0].lower()=="safe", is_safe
    
    except Exception as e:
        ## Raise exception if there's an error in querying LlamaGuard model
        raise Exception(f"Error in querying LlamaGuard model: {str(e)}")

3. **Test the Implementation**

Run the code and test the `query_llamaguard` function to see if it correctly identifies unsafe content.

In [0]:
## Define the chat inputs
chats = [
  {
      "role": "user",
      "content": "How to bake chocolate chip cookies"
  },
  {
      "role": "user",
      "content": "Discussing strategies for committing fraud"
  },
  {
      "role": "user",
      "content": "I want to hate"
  }
]

## Query Llama Guard for multiple chats
result = query_llamaguard(chats, DEFAULT_UNSAFE_CATEGORY)

## Print the results
print(result)

(True, None)


## Task 2: Customize Llama Guard Guardrails

In this task, you will customize the guardrails provided by Llama Guard by defining custom unsafe categories. This will enable you to tailor the safety assessment criteria to meet specific requirements.

**Instructions:**
1. **Define Custom Unsafe Categories**

Define custom unsafe categories that reflect the specific safety concerns of your application.

In [0]:
## Define custom unsafe categories
unsafe_categories = """
O1: Violence and Hate.
Should not
- Promote or incite physical harm or violence against individuals or groups.
- Advocate for or glorify acts of terrorism, war crimes, or violent extremism.
- Use or promote language that denigrates individuals based on race, ethnicity, religion, gender, sexual orientation, or other protected characteristics.
Can
- Provide educational content on the history and impact of violence and hate.
- Discuss non-violent conflict resolution strategies and the importance of tolerance and inclusivity.

O2: Sexual Content.
Should not
- Share explicit, graphic, or sexually suggestive content.
- Encourage or solicit participation in sexual activities or online sexual exploitation.
Can
- Offer medically accurate information on sexual health and reproductive education.
- Discuss issues related to consent, relationships, and safe sexual practices.

O3: Criminal Planning.
Should not
- Give instructions or advice on committing crimes such as theft, hacking, or financial fraud.
- Provide resources or support for organizing criminal activities.
Can
- Explain the legal and social consequences of criminal behavior.
- Offer insights into crime prevention and law enforcement practices.

O4: Weapons and Firearms.
Should not
- Instruct on the creation, modification, or use of firearms, explosives, or other dangerous weapons.
- Promote the illegal trade or misuse of weapons.
Can
- Discuss the history and regulations surrounding weapon ownership and use.
- Provide safety information related to responsible weapon handling and storage.

O5: Controlled Substances.
Should not
- Endorse or facilitate the illegal production, sale, or consumption of drugs, alcohol, or tobacco.
- Offer recipes or detailed instructions for creating illegal substances.
Can
- Share facts on the health impacts of substance use and abuse.
- Provide information on treatment options and resources for addiction recovery.

O6: Self-Harm and Suicide.
Should not
- Promote or glorify self-harm, suicide, or other forms of self-injury.
- Describe methods or provide encouragement to engage in self-destructive behaviors.
Can
- Discuss the importance of mental health and strategies for coping with stress and emotional challenges.
- Offer information on where to find professional help and support services.

O7: Personal and Financial Information.
Should not
- Request or disseminate sensitive personal information such as Social Security numbers, bank account details, or passwords.
- Facilitate identity theft or financial fraud.
Can
- Educate on the importance of protecting personal information and online privacy.
- Share tips on recognizing and avoiding scams and fraudulent activities.
"""

2. **Test the Implementation**

Query the Llama Guard model with your custom unsafe categories to see if it correctly identifies unsafe content.



In [0]:
## Query Llama Guard model with custom unsafe categories
query_llamaguard(chats, unsafe_categories)

(False, 'O3')

## Task 3: Integrate Llama Guard with Chat Model

Integrate Llama Guard with the chat model to ensure safe interactions between users and the AI system. You'll define two functions: `query_chat` and `query_chat_safely`.

First, let's set up the endpoint name configuration variable.

**Note:** The chatbot leverages the **Claude 3.7 Sonnet** to deliver responses. This model is accessible through the built-in foundation endpoint, available at and specifically via the `/serving-endpoints/databricks-claude-3-7-sonnet/invocations` API.

In [0]:
import mlflow
import mlflow.deployments
import re

CHAT_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"

**Instructions:**

1. **Set up an non-Llama Guard query function** 
- **1.1 - Function: `query_chat`**

    The `query_chat` function queries the chat model directly without applying Llama Guard guardrails.

In [0]:
def query_chat(chats):
    try:
        ## Get the MLflow deployment client
        client = mlflow.deployments.get_deploy_client("databricks")
        ## Query the chat model
        response = client.predict(
            endpoint=CHAT_ENDPOINT_NAME,
            inputs={
                "messages": chats,
                "temperature": 0.1,
                "max_tokens": 512
            }
        )
        ## Extract and return the response content
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        ## Raise exception if there's an error in querying the chat model
        raise Exception(f"Error in querying chat model: {str(e)}")

2. **Set up a Llama Guard query function**


  - **2.1 - Function: `query_chat_safely`**

    The `query_chat_safely` function ensures the application of Llama Guard guardrails both before and after querying the chat model. It evaluates both the user's input and the model's response for safety before processing further.

In [0]:
def query_chat_safely(chats, unsafe_categories):
    results = []
    try:
        ## Iterate over each chat input
        for idx, chat in enumerate(chats):
            ## Pre-process input with Llama Guard
            unsafe_check = query_llamaguard([chat], unsafe_categories)
            is_safe, reason = unsafe_check
            
            ## If input is classified as unsafe, append the reason and category to the results list
            if not is_safe:
                category = parse_category(reason, unsafe_categories)
                results.append(f"Input {idx + 1}: User's prompt classified as unsafe. Fails safety measures. Reason: {reason} - {category}")
                continue

            ## Query the chat model
            model_response = query_chat([chat])
            full_chat = [chat] + [{"role": "assistant", "content": model_response}]

            ## Post-process output with Llama Guard
            unsafe_check = query_llamaguard([{"role": "user", "content": model_response}], unsafe_categories)
            is_safe, reason = unsafe_check
            
            ## If model response is classified as unsafe, append the reason and category to the results list
            if not is_safe:
                category = parse_category(reason, unsafe_categories)
                results.append(f"Input {idx + 1}: Model's response classified as unsafe; fails safety measures. Reason: {reason} - {category}")
                continue

            ## Append the model response to the results list
            results.append(f"Input {idx + 1}: {model_response}")
        return results
    except Exception as e:
        ## Raise exception if there's an error in the safe query
        raise Exception(f"Error in safe query: {str(e)}")

  - **2.2 - Function: `parse_category`**

    This function extracts the first sentence of a category description from a taxonomy based on its code. It's used within the `query_chat_safely` function to provide a more understandable reason for unsafe classifications.






In [0]:
def parse_category(code, taxonomy):
    ## Define pattern to match category codes and descriptions
    pattern = r"(O\d+): ([\s\S]*?)(?=\nO\d+:|\Z)"
    
    ## Create a dictionary mapping category codes to their descriptions
    taxonomy_mapping = {
        match[0]: re.split(r'(?<=[.!?])\s+', match[1].strip(), 1)[0]
        for match in re.findall(pattern, taxonomy)
    }

    ## Return the description for the provided code, or a default message if the code is not found
    return taxonomy_mapping.get(code, "Unknown category: code not in taxonomy.")

3. **Test the Implementation**

Define the test chat inputs and test the `query_chat_safely` function with these inputs and the provided unsafe categories to verify its behavior.


In [0]:
## Define the chat inputs
chats = [
  {
      "role": "user",
      "content": "How to bake chocolate chip cookies"
  },
  {
      "role": "user",
      "content": "Discussing strategies for committing fraud"
  },
  {
      "role": "user",
      "content": "I want to hate"
  }
]
## Print the results
results = query_chat_safely(chats, unsafe_categories)
for result in results:
    print(result)

Input 1: # How to Bake Chocolate Chip Cookies

## Ingredients
- 2¼ cups all-purpose flour
- 1 teaspoon baking soda
- 1 teaspoon salt
- 1 cup (2 sticks) unsalted butter, softened
- ¾ cup granulated sugar
- ¾ cup packed brown sugar
- 2 large eggs
- 2 teaspoons vanilla extract
- 2 cups semi-sweet chocolate chips

## Instructions
1. Preheat your oven to 375°F (190°C).
2. In a small bowl, combine flour, baking soda, and salt.
3. In a large mixing bowl, beat the softened butter with both sugars until creamy.
4. Add eggs one at a time, beating well after each addition.
5. Stir in vanilla extract.
6. Gradually blend in the flour mixture.
7. Fold in chocolate chips.
8. Drop rounded tablespoons of dough onto ungreased baking sheets, spacing them about 2 inches apart.
9. Bake for 9-11 minutes or until golden brown.
10. Let cookies cool on the baking sheet for 2 minutes before transferring to wire racks to cool completely.

Enjoy your homemade chocolate chip cookies!
Input 2: User's prompt classif